# Q&A with multiple text files

With this notebook you can do question answering on three film scripts (Pulp fiction, Reservoir Dogs and Jackie Brown).

### Sources
https://www.youtube.com/watch?v=DXmiJKrQIvg



### Contents
0. Install packages
1. Settings
2. Download pdf's and convert to txt
3.

## 0. Install packages

In [1]:
!pip install langchain
!pip install unstructured
!pip install openai
!pip install chromadb
!pip install Cython
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 1.9 MB/s eta 0:00:00


  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
weaviate-client 3.0.0 requires requests<2.26.0,>=2.23.0, but you have requests 2.31.0 which is incompatible.
datasets 2.11.0 requires tqdm>=4.62.1, but you have tqdm 4.61.2 which is incompatible.
conda-repo-cli 1.0.27 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.27 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.27 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.


In [2]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


## 1. Settings

In [3]:
#Load Required Packages
#from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader

In [4]:
# Get your API keys from openai, you will need to create an account. 
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
import config
os.environ["OPENAI_API_KEY"] = config.openai_key

## 2. Download pdf's and convert to txt 

In [5]:
import wget
wget.download('https://assets.scriptslug.com/live/pdf/scripts/pulp-fiction-1994.pdf')
wget.download('https://assets.scriptslug.com/live/pdf/scripts/reservoir-dogs-1992.pdf')
wget.download('https://assets.scriptslug.com/live/pdf/scripts/jackie-brown-1997.pdf')

100% [........................................................] 198596 / 198596

'jackie-brown-1997 (1).pdf'

In [6]:
import glob
my_pdfs = glob.glob('*.pdf')
my_pdfs

['reservoir-dogs-1992 (1).pdf',
 'jackie-brown-1997 (1).pdf',
 'pulp-fiction-1994.pdf',
 'jackie-brown-1997.pdf',
 'pulp-fiction-1994 (1).pdf',
 'reservoir-dogs-1992.pdf']

In [7]:
#THE one script to make one long txt file (by adding )
from PyPDF2 import PdfReader
import os

for i in range(len(my_pdfs)):
    reader = PdfReader(my_pdfs[i])
    number_of_pages = len(reader.pages)
    file_name, ext = os.path.splitext(my_pdfs[i])
    
    textfile = open(file_name+".txt", "w")

    for j in range (number_of_pages):
        page = reader.pages[j]
        textfile.write(page.extract_text())
        textfile.write('}\n')
    textfile.close()

In [8]:
import glob
my_txts = glob.glob('*.txt')
my_txts

['jackie-brown-1997 (1).txt',
 'My Clippings.txt',
 'pulp-fiction-1994.txt',
 'reservoir-dogs-1992 (1).txt',
 'reservoir-dogs-1992.txt',
 'state_of_the_union.txt',
 'jackie-brown-1997.txt',
 'pulp-fiction-1994 (1).txt']

## 3. Load multiple txt files with Langchain into Chroma vectorstore

In [9]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()

In [10]:
documents

[Document(page_content='JACKIE BROWN\nScreenplay by\nQuentin Tarantino\nBased on the Novel by Elmore Leonard}\nOPENING CREDITS\nINT. LOS ANGELES INTERNATIONAL AIRPORT – DAY\nWe hear the rhythm of funky seventies SOUL MUSIC .\nThen SHE steps into FRAME .\nShe is JACKIE BROWN , a stewardess dressed in her CABO AIR  uniform. (A little shuttle\nairline that flies from Los Angeles to Cabo San Lucas. Approximate flight time: forty five\nminutes)\nJackie stands still as a people-mover slowly inches her through the airport. The CREDITS\nAPEAR  and DISAPPEAR  in front of her.\nJackie Brown is a very attractive black woman in her mid forties, though she looks like she\'s in\nher mid-thirties.\nThe people-mover reaches the end of the line, she steps off.\nShe breezes through Customs and we follow her with a STEDICAM  as she strides through the\nairport... She gets to her gate disappears inside the plane for a moment comes back out sans\nflight bag picks up the microphone.\nJACKIE\n(into mike)\nFl

In [11]:
#splitting the text into
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1133

### What is a chroma vector store? 
Chroma as vectorstore to index and search embeddings


There are three main steps going on after the documents are loaded:

- Splitting documents into chunks

- Creating embeddings for each document

- Storing documents and embeddings in a vectorstore


In [12]:
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(documents=texts)

Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


## 4. Q&A with the documents

In [13]:
index.query_with_sources('Who is vincent vega?')

NameError: name 'index' is not defined

In [15]:
pwd

'/Users/michielbontenbal/Documents/GitHub/chat_with_documents'

In [16]:
pdf_folder_path = '/Users/michielbontenbal/Documents/GitHub/chat_with_documents'
os.listdir(pdf_folder_path)

['jackie-brown-1997 (1).txt',
 '5. Langchain full tutorial on question_answering.ipynb',
 'LangchainPDF (unstructured_inference issue).ipynb',
 '6. Vector databases with Chroma.ipynb',
 '.chroma',
 '.DS_Store',
 'config.py',
 'reservoir-dogs-1992 (1).pdf',
 'My Clippings.txt',
 'pulp-fiction-1994.txt',
 'LICENSE',
 'Semantic Search on wikipedia embeddings with Cohere.ipynb',
 'weaviate.ipynb',
 'Multiple pdfs (detectron2 issue).ipynb',
 'hnswlib',
 'jackie-brown-1997 (1).pdf',
 'PDF',
 'vector_db_qa.ipynb',
 '__pycache__',
 'reservoir-dogs-1992 (1).txt',
 'pulp-fiction-1994.pdf',
 'README.md',
 '2. Getting started with langchain - question answering on 1 txt file.ipynb',
 'reservoir-dogs-1992.txt',
 '.gitignore',
 'jackie-brown-1997.pdf',
 '1. Getting started with the OpenAI API.ipynb',
 'pulp-fiction-1994 (1).pdf',
 'db',
 'LangchainPDF (unstructured issue).ipynb',
 '.ipynb_checkpoints',
 'reservoir-dogs-1992.pdf',
 '.git',
 'Multi_doc_retriever_with Langchain_Quentin Tarantino script

In [17]:
# location of the pdf file/files. 
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
index = VectorstoreIndexCreator().from_loaders(loaders)
index

NameError: name 'UnstructuredPDFLoader' is not defined

In [ ]:
index.query('How was the GPT4all model trained?')

' The GPT4all model was trained with LoRA (Hu et al., 2021) on the 437,605 post-processed examples for four epochs.'

In [ ]:
index.query_with_sources('How was the GPT4all model trained?')

{'question': 'How was the GPT4all model trained?',
 'answer': ' The GPT4all model was trained with LoRA on 437,605 post-processed examples for four epochs. The data was collected using the GPT-3.5-Turbo OpenAI API and was curated to ensure a diverse distribution of prompt topics and model responses.\n\n',
 'sources': '/content/gdrive/My Drive/data_2/2023_GPT4All_Technical_Report.pdf'}

In [ ]:
index.query_with_sources('Who wrote the lip sync paper? ')

{'question': 'Who wrote the lip sync paper? ',
 'answer': ' The lip sync paper was written by K R Prajwal, Vinay P. Namboodiri, Rudrabha Mukhopadhyay, and C V Jawahar.\n',
 'sources': '/content/gdrive/My Drive/data_2/2008.10010.pdf'}

In [ ]:
index.query_with_sources('How was the GPT4all model trained?')

{'question': 'How was the GPT4all model trained?',
 'answer': ' The GPT4all model was trained with LoRA on 437,605 post-processed examples for four epochs. Detailed model hyper-parameters and training code can be found in the associated repository and model training log.\n',
 'sources': '/content/gdrive/My Drive/data_2/2023_GPT4All_Technical_Report.pdf'}

In [ ]:
index.query_with_sources('Who wrote the lip sync paper? ')

{'question': 'Who wrote the lip sync paper? ',
 'answer': ' The lip sync paper was written by K R Prajwal, Vinay P. Namboodiri, Rudrabha Mukhopadhyay, and C V Jawahar.\n',
 'sources': '/content/gdrive/My Drive/data_2/2008.10010.pdf'}

## Disclaimer:
Note: OpenAI provides a free API key for initial testing. Once you move to a paid subscription, calling the API in the way demonstrated in this example will incur monetary charges. Refer to OpenAI's pricing information for details.

Be aware that information, such as files to train OpenAI's LLM can become public if applied in the way this demo demonstrates. Refer to OpenAI's usage policy for details.

Do not use for actual tax filing purposes. This demo is for educational purposes only and for demonstrating machine learning methods. The author makes no claims that the outcomes shown here or any outcomes that could be produced by this method are accurate or reliable.